### Hello! Welcome to my assignment on Clustering with Spatial data of Coursera Capstone Project, week 3. 
This notebook will be used primarily to scrape a Wikipedia page containing data on all neighborhoods in the city of Toronto (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) and, after all wrangling routines are done, the prepared dataset will be read in a pandas dataframe in order to be used in the Foursquare API for queries. 

In [2]:
import pandas as pd
import numpy as np
import geocoder
import bs4
import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

print("No errors found - Everything successfully loaded!")

No errors found - Everything successfully loaded!


### PART 1 - Scrapping neighborhood data from Wikipedia and cleaning it


In [3]:
webpage = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(webpage,'lxml') #parsing data 
table_neighbor=[] #creating empty table instance
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':#Ignoring cells with a borough that is Not assigned
        pass
    else:
        cell['PostalCode']=row.p.text[:3]
        cell['Borough']=(row.span.text).split('(')[0]
        cell['Neighborhood']= (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ') 
        table_neighbor.append(cell)


In [4]:
df=pd.DataFrame(table_neighbor)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A','East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business','EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto','MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
#df.info
df


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### PART 2 - Adding Geospatial data to the dataframe

In [6]:
#download geographical coordinates of the neighborhoods using the Geocoder package
lat_lon = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
lat_lon.head()
lat_lon.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df_merged = pd.merge(df,lat_lon,on='PostalCode')
print(df_merged.shape)
df_merged.sample(n=10)


(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
54,M4M,East Toronto,Studio District,43.659526,-79.340923
46,M3L,North York,Downsview West,43.739015,-79.506944
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
79,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817
1,M4A,North York,Victoria Village,43.725882,-79.315572
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
37,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
21,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
45,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714
